In [266]:
import json
import pandas as pd
from coinmarketcap import Market
# function that checks for NaN in a pandas df
def check_nan(df):
    count = 0
    for i,j in enumerate(df.isnull().sum()):
        if j != 0:
            print(df.columns[i], "contains {} NaN".format(j))
            count += 1
    if count == 0:
        print("no NaN")


coinmarketcap = Market()
# the output of below call is byte so we need to convert it to str
a = coinmarketcap.ticker()
# conversion bytecode to str
json_thing = a.decode()
json_thing = json.loads(json_thing)
df = pd.DataFrame(json_thing)
df.last_updated = pd.to_datetime(df.last_updated, unit='s')
del df['available_supply']
del df['id']
del df['percent_change_1h']
del df['percent_change_7d']
del df['name']
del df['total_supply']

df = df[['24h_volume_usd', 'symbol', 'last_updated', 'market_cap_usd',
       'percent_change_24h', 'price_btc', 'price_usd', 'rank']]
check_nan(df)
df.dropna(inplace=True)
check_nan(df)
# df.sort_values('24h_volume_usd', ascending=False, inplace=True)
json_thing

24h_volume_usd contains 36 NaN
last_updated contains 1 NaN
market_cap_usd contains 180 NaN
percent_change_24h contains 27 NaN
price_btc contains 1 NaN
price_usd contains 1 NaN
no NaN


[{'24h_volume_usd': '1730490000.0',
  'available_supply': '16355600.0',
  'id': 'bitcoin',
  'last_updated': '1495925651',
  'market_cap_usd': '33512951512.0',
  'name': 'Bitcoin',
  'percent_change_1h': '2.71',
  'percent_change_24h': '-6.25',
  'percent_change_7d': '-0.56',
  'price_btc': '1.0',
  'price_usd': '2049.02',
  'rank': '1',
  'symbol': 'BTC',
  'total_supply': '16355600.0'},
 {'24h_volume_usd': '1219430000.0',
  'available_supply': '91982467.0',
  'id': 'ethereum',
  'last_updated': '1495925664',
  'market_cap_usd': '14251763422.0',
  'name': 'Ethereum',
  'percent_change_1h': '3.44',
  'percent_change_24h': '-3.36',
  'percent_change_7d': '20.23',
  'price_btc': '0.0754323',
  'price_usd': '154.94',
  'rank': '2',
  'symbol': 'ETH',
  'total_supply': '91982467.0'},
 {'24h_volume_usd': '262404000.0',
  'available_supply': '38249335400.0',
  'id': 'ripple',
  'last_updated': '1495925650',
  'market_cap_usd': '8302018249.0',
  'name': 'Ripple',
  'percent_change_1h': '7.21'

In [140]:
# limit = 2000 <> rows 2000
# from 2014-10-11 03:00:00
# to 2015-01-02 10:00:00

print(r.json()['TimeFrom'])
print(r.json()['TimeTo'])

1413021600
1420192800


In [179]:
import datetime
print("Date From",datetime.datetime.fromtimestamp(r.json()['TimeFrom']))
print("Date To", datetime.datetime.fromtimestamp(r.json()['TimeTo']))

date_from  = datetime.datetime(2015, 1, 2, 10, 00, 00).timestamp()
date_from

Date From 2014-10-11 04:00:00
Date To 2015-01-02 10:00:00


1420192800.0

In [170]:
datetime.datetime.fromtimestamp(1420196399)

datetime.datetime(2015, 1, 2, 10, 59, 59)

In [268]:
import datetime
import time
dt = parser.parse("2014-10-11 04:00:00")
time.mktime(dt.timetuple())

1412982000.0

In [160]:
from dateutil import parser
dt = parser.parse("2014-10-11 04:00:00")

In [265]:
# Hourly data from cryprocompare
import requests
from IPython.display import display

# generic url for data extraction from cryptocompare.com's API
url = 'https://min-api.cryptocompare.com/data/{0}?fsym={1}&tsym={2}&limit={3}&toTs={4}{5}'

# parameters for the url
# data resolution {0}
hourly_data = "histohour"
minute_data = "histominute"
# currency from {1}
curr_from = "BTC"
# currency to {2}
curr_to = "USD"
# limit of hours with maximum 2000 (1992 is 83 days) {3}
hour_limit = 2000

############################################################
# calculating and converting time for data extraction {4}
date_from_datetime  = datetime.datetime(2014, 10, 11, 3, 0)
print(date_from_datetime)

# enter date_to that is <= than 83 daya from date_from 
date_to_datetime = datetime.datetime(2015, 1, 2, 10, 0)
print(date_to_datetime)

# checking that the entered dates have no more than 2000 hr between them
delta_in_hours = int(((date_to_datetime - date_from_datetime).total_seconds() / 3600))
print("There are {0} hrs or {1:.2f} days between the dates {2} and {3}".format(delta_in_hours, delta_in_hours / 24, \
      date_from_datetime, date_to_datetime))
limit_hours = 2000
assert (delta_in_hours < limit_hours), \
"There must be <= {0} hrs between dates, but the current value is {1}. Change the dates interval"\
.format(limit_hours, delta_in_hours)
    
# calculating the the date_to by adding 1999 hr to the date_from
# date_to = (date_from_datetime + datetime.timedelta(hours=2000-1))
date_to_timestamp = date_to_datetime.timestamp()
print(date_to_timestamp)
############################################################

# coin exchange name, if not used, then CCAGG  {5}
exchange =  ""#"&e=Kraken" (default exchange is CCAGG)



r = requests.get(url.format(hourly_data, curr_from, curr_to, hour_limit-1, date_to_timestamp, exchange))
df = pd.DataFrame(r.json()['Data'])
df.time = pd.to_datetime(df.time, unit='s')
if exchange == "": exchange = "CCAGG"
print("Prices from Coin Exchange {}".format(exchange))
display(df.head(3))
print("##########################")
display(df.tail(3))




2014-10-11 03:00:00
2015-01-02 10:00:00
There are 1999 hrs or 83.29 days between the dates 2014-10-11 03:00:00 and 2015-01-02 10:00:00
1420192800.0
Prices from Coin Exchange CCAGG


,close,high,low,open,time,volumefrom,volumeto
0,361.32,369.22,359.65,366.44,2014-10-11 03:00:00,629.93,230371.85
1,364.00,365.39,360.99,361.32,2014-10-11 04:00:00,546.04,198449.22
2,362.94,364.09,361.23,364.00,2014-10-11 05:00:00,547.33,199066.29


##########################


,close,high,low,open,time,volumefrom,volumeto
1997,314.77,315.44,313.46,314.07,2015-01-02 08:00:00,524.55,165312.48
1998,314.35,315.38,313.88,314.77,2015-01-02 09:00:00,392.10,123428.71
1999,314.05,315.01,313.49,314.35,2015-01-02 10:00:00,520.43,163555.71
